In [1]:
from DiffFlatQuad.robot import PlanerQuadrotor

pygame 2.1.2 (SDL 2.0.16, Python 3.8.10)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
robot = PlanerQuadrotor(rendering=False)

In [3]:
robot.getSymbolicF()

Matrix([
[x_2],
[  0],
[x_4],
[ -g],
[x_6],
[  0]])

In [4]:
robot.symbolic_input

Matrix([
[u_1],
[u_2]])

In [5]:
f = robot.getSymbolicF()
g = robot.getSymbolicG()
x = robot.symbolic_state
g1 = g[:,0]
g2 = g[:,1]


In [6]:
def lieDerivative(a, b, x):
    """ 
    returns the L_a(b)
    """
    return b.jacobian(x)*a

def lieBracket(a,b,x):
    return lieDerivative(a, b, x)-lieDerivative(b, a, x)

In [7]:
c = lieBracket(g1,f, x)
c

Matrix([
[     sin(x_5)/m],
[-x_6*cos(x_5)/m],
[     cos(x_5)/m],
[ x_6*sin(x_5)/m],
[              0],
[              0]])

In [43]:
c = lieBracket(c,f, x)
c

Matrix([
[-x_5*(x_5*(x_5*cos(x_5)/m + 2*sin(x_5)/m) - x_5*(-x_5*(-x_5*sin(x_5)/m + 3*cos(x_5)/m) - 2*x_5*cos(x_5)/m - 4*sin(x_5)/m) + x_5*(2*x_5*(-x_5*sin(x_5)/m + 3*cos(x_5)/m) - x_5*(-x_5*(-x_5*cos(x_5)/m - 4*sin(x_5)/m) + 3*x_5*sin(x_5)/m - 9*cos(x_5)/m) + 4*x_5*cos(x_5)/m + 8*sin(x_5)/m) - x_5*(-4*x_5*(-x_5*sin(x_5)/m + 3*cos(x_5)/m) + 2*x_5*(-x_5*(-x_5*cos(x_5)/m - 4*sin(x_5)/m) + 3*x_5*sin(x_5)/m - 9*cos(x_5)/m) - x_5*(3*x_5*(-x_5*cos(x_5)/m - 4*sin(x_5)/m) - x_5*(-x_5*(x_5*sin(x_5)/m - 5*cos(x_5)/m) + 4*x_5*cos(x_5)/m + 16*sin(x_5)/m) - 9*x_5*sin(x_5)/m + 27*cos(x_5)/m) - 8*x_5*cos(x_5)/m - 16*sin(x_5)/m) + x_5*sin(x_5)/m - cos(x_5)/m) - x_5*(4*x_5*(x_5*cos(x_5)/m + 2*sin(x_5)/m) + x_5*(2*x_5*cos(x_5)/m + 4*sin(x_5)/m) - 3*x_5*(-x_5*(-x_5*sin(x_5)/m + 3*cos(x_5)/m) - 2*x_5*cos(x_5)/m - 4*sin(x_5)/m) - x_5*(-x_5*(-2*x_5*sin(x_5)/m + 6*cos(x_5)/m) - x_5*(-x_5*sin(x_5)/m + 3*cos(x_5)/m) - 6*x_5*cos(x_5)/m - 12*sin(x_5)/m) + 2*x_5*(2*x_5*(-x_5*sin(x_5)/m + 3*cos(x_5)/m) - x_5*(-x_5*

In [8]:
lieBracket(g2,f, x)

Matrix([
[  0],
[  0],
[  0],
[  0],
[1/J],
[  0]])

In [48]:
f

Matrix([
[ x_2],
[   0],
[ x_4],
[-g*m],
[ x_5],
[   0]])

In [14]:
f.row_join(g1)

Matrix([
[x_2,          0],
[  0, sin(x_5)/m],
[x_4,          0],
[ -g, cos(x_5)/m],
[x_6,          0],
[  0,          0]])

In [31]:
def isInDist(dist, vec):
    """
    Is vec in distribution dist?
    """
    d = dist[0].copy()
    for i in range(len(dist)-1):
        d = d.row_join(dist[i+1])
    rank1 = d.rank()
    d = d.row_join(vec)
    rank2 = d.rank()
    if rank2 > rank1:
        return False
    else:
        return True
    
def getDistRank(dist):
    """
    return the rank of distribution spanned by list of vecs in dist
    """
    d = dist[0].copy()
    for i in range(len(dist)-1):
        d = d.row_join(dist[i+1])
    return d.rank()

def getSmallestInvariantDistribution(dist, vec_fields, x):
    """
    Returns the smallest distribution invariant to vectors in the 
    vec_fields list and containing distribution spanned by the 
    vector fields in dist list. 
    """
    running = True
    result = []
    result +=dist
    while running:
        added_something = False
        for vec1 in result:
            for vec2 in vec_fields:
                vec3 = lieBracket(vec1, vec2, x)
                if not isInDist(result, vec3):
                    result +=[vec3]
                    added_something = True
        if not added_something:
            running = False
    
    return result
                    
        

In [34]:
isInDist([g2,g2,f], f)

True

In [37]:
result = getSmallestInvariantDistribution([g1,g2], [f,g1,g2],x)

In [38]:
len(result)

6